<a href="https://colab.research.google.com/github/wbandabarragan/computational-physics/blob/main/fisica-interactiva/corrimiento_cosmologico_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corrimiento al Rojo Cosmológico

La expansión del Universo causa corrimientos al rojo cosmológicos ($z$) de líneas espectrales que en el presente ($z=0$) se encuentran en sus sistemas de referencia en reposo. La relación entre las longitudes de onda observada $\lambda_{\text{obs}}$ y emitida por un objeto lejano $\lambda_{\text{em}}$ ubicado a un corrimiento al rojo $z$ está dada por:

$\lambda_{\text{obs}} = \lambda_{\text{em}} (1 + z)$

La frecuencia $\nu$ es inversamente proporcional a la frecuencia, entonces:

$\nu_{\text{obs}} = \frac{\nu_{\text{em}}}{1 + z}$

Además la edad del Universo a un $z$ determinado se puede calcular integrando el inverso del parámetro de Hubble:

$t(z) = \int_{z}^{\infty} \frac{1}{(1+z') H(z')} d{z'}$

El factor de expansión cosmológica $a(t)$ está relacionado al corrimiento al rojo a través de:

$a = \frac{1}{1+z}$

En el modelo \(\Lambda\)-CDM, el parámetro de Hubble $H(z)$ se expresa como:

$H(z) = H_0 \sqrt{\Omega_m (1+z)^3 + \Omega_r (1+z)^4 + \Omega_\Lambda}$

Donde:

- $H_0$: constante de Hubble a tiempo presente.
  
- $\Omega_m$: Parámetro de densidad de materia
  
- $\Omega_r$: Parámetro de densidad de radiación

- $\Omega_\Lambda$: Parámetro de densidad de energía oscura.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Checkbox, VBox
from scipy.integrate import quad

# Parámetros cosmológicos (modelo estándar)
H0 = 70.0  # Constante de Hubble en km/s/Mpc
Omega_m = 0.27  # Parámetro de densidad de materia oscura + bariónica
Omega_r = 0.0  # Parámetro de densidad de radiación (se puede tomar como cero)
Omega_Lambda = 0.73  # Parámetro de densidad de energía oscura

# Constantes de conversión
H0_in_s = H0 * 3.24078e-20  # Convertir H0 de km/s/Mpc a 1/s
Mpc_in_km = 3.085677581e19  # 1 Mpc en kilómetros
s_to_Gyr = 3.16888e-17  # 1 segundo en Gyr

# Parámetro de Hubble como función del corrimiento al rojo
def H_z(z, H0, Omega_m, Omega_r, Omega_Lambda):
    """
    Calcula H(z) usando el modelo Lambda-CDM.
    Entradas: z -> Corrimiento al rojo, H0 -> Constante de Hubble (en km/s/Mpc),
              Omega_m -> Densidad de materia oscura/bariónica,
              Omega_r -> Densidad de radiación,
              Omega_Lambda: Densidad de energía oscura
    Salida: H(z) en km/s/Mpc
    """
    return H0 * np.sqrt(Omega_m * (1 + z)**3 + Omega_r * (1 + z)**4 + Omega_Lambda)

# Función para calcular el integrando para la edad del Universo
def integrand(z, H0, Omega_m, Omega_r, Omega_Lambda):
    """
    Calcula la integral 1 / [(1 + z) H(z)] para hallar la edad del Universo.
    """
    return 1 / ((1 + z) * H_z(z, H0, Omega_m, Omega_r, Omega_Lambda))

# Edad del Universo al corrimiento al rojo z
def age_of_universe(z, H0=H0_in_s, Omega_m=Omega_m, Omega_r=Omega_r, Omega_Lambda=Omega_Lambda):
    """
    Calcula la edad del Universo en Gyr para un corrimiento al rojo dado z.
    Entradas: z -> Corrimiento al rojo
              H0, Omega_m, Omega_r, Omega_Lambda -> Parámetros cosmológicos
    Salida: Edad del Universo en Gyr
    """
    # Realiza la integración desde z hasta infinito
    age, _ = quad(integrand, z, np.inf, args=(H0, Omega_m, Omega_r, Omega_Lambda))
    return age * s_to_Gyr  # Convierte la edad de segundos a Gyr

# Función para calcular H(z) y la edad del Universo
def calculate_Hz_and_age(z):
    """
    Calcula H(z) y la edad del Universo para un corrimiento al rojo dado z.
    Entradas:
        z: Corrimiento al rojo
    Salida:
        Hz: Parámetro de Hubble en km/s/Mpc
        age: Edad del Universo en Gyr
    """
    # Calcula H(z)
    Hz = H_z(z, H0, Omega_m, Omega_r, Omega_Lambda)

    # Calcula la edad del Universo
    age = age_of_universe(z, H0=H0_in_s, Omega_m=Omega_m, Omega_r=Omega_r, Omega_Lambda=Omega_Lambda)

    return Hz, age

# Línea Lyman alpha sintética
def linea_espectral(x_val, mu, sigma, n_points):
    """
    This return the values for a normal distro.
    Inputs: x -> x axis, mu -> mean, sigma -> std. dev., n_points -> number of points for noise
    Output: y
    """

    # Función Gaussiana
    coef_g = 1./(sigma*(np.sqrt(2*np.pi)))

    y_val = coef_g*np.exp(-0.5*(x_val - mu)**2/sigma**2)

    # Semilla para generador aleatorio
    np.random.seed(77)

    # Añadiendo ruido
    n_val = 1.e-4*np.random.normal(0., 1., size = (n_points,))

    # Añadiendo una línea de base
    b_val = 0.1

    y = y_val + n_val + b_val

    return y

# Corrimiento al rojo cosmológico
def z_cosmologico(l_0, z):
    """
    Función que calcula el corrimiento al rojo cosmológico de una línea
    en un sistema de referencia en reposo (z=0).
    Entrada: l_0 -> longitud de onda de la línea espectral base
    Salida: l_obs -> longitud de onda observada.
    """
    l_obs = l_0*(1. + z)
    return l_obs

# Función de ploteo
def plot_lines(z, show_Lyman_alpha, show_H_alpha):
    # Longitudes de onda de Lyman-alpha y H-alpha
    l_lyman_alpha = 1216  # Lambda para Lyman-alpha (in Angstroms)
    l_h_alpha = 6563  # Lambda para H-alpha (in Angstroms)
    sigma_1 = 50  # Desviación estándar de la línea espectral Gaussiana
    n_points = 10000  # Número de puntos a ser considerados

    # Ejemplo de uso
    Hz, age = calculate_Hz_and_age(z)

    # Eje lambda
    x = np.linspace(0, 16000, n_points)

    # Gráfica
    plt.figure(figsize=(12,5))

    plt.title(f"La constante de Hubble H(z) es {Hz:.2f} km/s/Mpc y la edad del Universo es {age:.2f} Gyr para z = {z}")

    # Rango visible del espectro (400 nm a 700 nm)
    visible_min = 4000  # 400 nm en Angstroms
    visible_max = 7000  # 700 nm en Angstroms

    plt.fill_between(x, 0, 1, where=(x >= visible_min) & (x <= visible_max), color='pink', alpha=0.5, label="Rango Visible")

    # Label the visible spectrum
    plt.axvline(visible_min, color='black', linestyle='--')
    plt.axvline(visible_max, color='black', linestyle='--')

    # Lyman Alpha
    if show_Lyman_alpha:
        # Lyman alpha profile
        y_lyman = linea_espectral(x, l_lyman_alpha, sigma_1, n_points)
        # Shift the Lyman-alpha line according to the redshift
        l_obs_lyman = z_cosmologico(l_lyman_alpha, z)
        y_obs_lyman = linea_espectral(x, l_obs_lyman, sigma_1, n_points)
        plt.plot(x, y_lyman, color = "red", label=r"Lyman $\alpha$ en z=0.0")
        plt.plot(x, y_obs_lyman, color = "blue", label=rf"Lyman $\alpha$ en z={z}")

    # H Alpha
    if show_H_alpha:
        # H-alpha profile
        y_h_alpha = linea_espectral(x, l_h_alpha, sigma_1, n_points)
        # Shift the H-alpha line according to the redshift
        l_obs_h_alpha = z_cosmologico(l_h_alpha, z)
        y_obs_h_alpha = linea_espectral(x, l_obs_h_alpha, sigma_1, n_points)
        plt.plot(x, y_h_alpha, color = "brown", label=r"H_\alpha en z=0")
        plt.plot(x, y_obs_h_alpha, color = "darkblue", label=f"H Alpha en z={z}")

    # Etiquetas
    plt.grid(True)
    plt.xlim(0, 16000)
    plt.ylim(0.095, 0.11)
    plt.xlabel(r"$\lambda\,[\text{Angstroms}]$")
    plt.ylabel(r"$N$")
    plt.legend(loc = 4)
    plt.show()

# Controles interactivos para z y cajas de opciones para las líneas
z_scroll = FloatSlider(value=0, min=0, max=10, step=0.1, description=r'$z$')

lyman_c = Checkbox(value=True, description=rf"Muestra Ly$\alpha$ a $\lambda_0=1216\,\rm A$")
halph_c = Checkbox(value=False, description=rf"Muestra H$\alpha$ a $\lambda_0=6563\,\rm A$")

# Display the interactive widgets
interact(plot_lines, z=z_scroll , show_Lyman_alpha=lyman_c, show_H_alpha=halph_c);


interactive(children=(FloatSlider(value=0.0, description='$z$', max=10.0), Checkbox(value=True, description='M…